In [ ]:
import time
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from new_sort.sort import Sort
import uuid
import threading
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()
parent_dir = os.getenv("PARENT_DIR")

class ObjectTracker:
    def __init__(self):
        self.tracker = Sort()
        self.custom_track_ids = {}

    def update(self, detections):
        return self.tracker.update(np.array(detections))

class CameraStream:
    def __init__(self, rtsp_url,camera_ip, camera_id, model_path,thread_id):
        self.thread_id = thread_id
        self.rtsp_url = rtsp_url
        self.camera_ip = camera_ip
        self.camera_id = camera_id
        self.model = YOLO(model_path).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(device)
        self.orig_w, self.orig_h = 1280, 920
        self.resized_w = int(os.getenv("RESIZED_RESOLUTION_WIDTH"))
        self.resized_h = int(os.getenv("RESIZED_RESOLUTION_HEIGHT"))
        self.frames = []
        self.tracker = [ObjectTracker() for i in self.rtsp_url]
        self.class_list = [
            "pothole",
            "crack",
            "manholes",
            "broken divider",
            "broken road side",
            "damaged drain",
            "open drain",
            "vegetation",
        ]
    def process_frame_batch(self, frames):
        resized_frames = [cv2.resize(frame, (self.resized_w // 32 * 32, self.resized_h // 32 * 32)) for frame in frames]
        frames_tensor = (
            torch.from_numpy(np.stack(resized_frames))
            .permute(0, 3, 1, 2)
            .float()
            .to(self.model.device)
            / 255.0
        )
        with torch.no_grad():
            return self.model(frames_tensor, device=self.model.device)

    def track_objects(self, frames, batch_results, frame_time):
        tracked_frames = []
        current_track_ids = []

        for frame, result in zip(frames, batch_results):
            detections, labels, confs = [], [], []
            scale_x, scale_y = self.orig_w / self.resized_w, self.orig_h / self.resized_h

            if result.masks:
                for mask, box in zip(result.masks.xy, result.boxes):
                    x1, y1, x2, y2 = (box.xyxy[0].cpu().numpy() * np.array([scale_x, scale_y, scale_x, scale_y])).astype(int)
                    score = box.conf[0].item()
                    label = self.model.names[int(box.cls[0])]
                    detections.append([x1, y1, x2, y2, score, int(box.cls[0])])
                    labels.append(label)
                    confs.append(score)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            if len(detections) == 0:
                continue
            tracks = self.tracker[0].update(detections)

            for i, track in enumerate(tracks):
                track_id = int(track[4])
                x1, y1, x2, y2 = map(int, track[:4])
            tracked_frames.append(frame)

        return tracked_frames, current_track_ids

    def generate_custom_track_id(self, label, confidence):
        return f"{label}_{confidence:.2f}_{uuid.uuid4()}"
    
    def connect_stream(self, url:str):
        cap = cv2.VideoCapture(url)
        while not cap.isOpened():
            print(f"Failed to connect. Retrying in 5 seconds...")
            time.sleep(5)
            cap = cv2.VideoCapture(url)  # Re-attempt connection
        print(f"Successfully connected!")
        return cap

    def run(self, batch_size = 20):
        print(self.rtsp_url)
        caps = [self.connect_stream(url = f"{video_url}") for video_url in self.rtsp_url]

        # caps = [cv2.VideoCapture(self.rtsp_url[0])]
        for c in caps:
            if not c.isOpened():
                print(f"Error opening video file: {self.rtsp_url[0]}")
                return
        frames = []
        start_time = time.time()
        interval = 120  # 2 min
        t1 = time.time()
        while True:
            for cap in caps:
                ret, frame = cap.read()
                if not ret:
                    break
                frame_time = time.time()
                frames.append(frame)
                # cv2.imshow("test",frame)
            if time.time() - start_time >= interval:
                cv2.imwrite(f"{parent_dir}/data/municipal/{frame_time}_{self.camera_id}.jpeg", frame)
                start_time = time.time()

            if len(frames) >= batch_size:  # Process in batches of 8
                batch_results = self.process_frame_batch(frames)
                tracked_frames, track_id_list = self.track_objects(frames, batch_results, frame_time)
                for tracked_frame in tracked_frames:
                    tracked_frame = cv2.resize(tracked_frame, (640, 480))
                    print(self.camera_id)
                    # cv2.imwrite("/home/annone/ai/data/img.jpeg",tracked_frame)
                    # cv2.imshow(f"Tracked Frame", tracked_frame)  
                    if cv2.waitKey(1) & 0xFF == ord("q"):
                        break
                frames = []

            print(self.thread_id)
        t2 = time.time()
        print(t2-t1, self.camera_ip)
        cap.release()
        cv2.destroyAllWindows()

class MultiCameraProcessor:
    def __init__(self, cam_rtsp_urls,cam_ids,cam_ips, model_path):
        self.cam_rtsp_urls = cam_rtsp_urls
        self.cam_ids = cam_ids
        self.cam_ips = cam_ips
        self.model_path = model_path

    def start_streams(self):
        threads = []
        for i in range(1):
            camera_stream = CameraStream(rtsp_url=cam_rtsp_urls, camera_ip= self.cam_ips,camera_id= self.cam_ids,model_path= self.model_path, thread_id=i)
            thread = threading.Thread(target=camera_stream.run)
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()

if __name__ == "__main__":
    cam_rtsp_urls = os.getenv("CAM_RTSPS").split(',')
    cam_ids = os.getenv("CAM_IDS").split(",")
    cam_ips = os.getenv("CAM_IPS").split(",")
    model_path = f"{os.getenv('PARENT_DIR')}/models/obj_seg_02.pt"
    camera_stream = CameraStream(rtsp_url=cam_rtsp_urls,camera_ip=cam_ips,camera_id=cam_ids,model_path=model_path, thread_id=1)
    thread = threading.Thread(target=camera_stream.run)
    thread.start()
    thread.join()
    # processor = MultiCameraProcessor(cam_rtsp_urls= cam_rtsp_urls,cam_ids=cam_ids,cam_ips=cam_ips,model_path=model_path)
    # processor.start_streams()


In [ ]:
import geocoder

def get_location():
    # Get location based on IP
    g = geocoder.ip('me')
    if g.ok:
        return g.latlng  # returns [latitude, longitude]
    else:
        return "Location could not be determined"

location = get_location()
print("Current location (latitude, longitude):", location)


In [ ]:
import cv2
import torch
from ultralytics import YOLO
from datetime import datetime
from roboflow import Roboflow
rf = Roboflow(api_key="o9xQlct9Wr77cZXqyV17")
project = rf.workspace().project("muncipal-detect")
model = project.version("3").model
# Load YOLOv8 segmentation model
# model = YOLO("yolov8n-seg.pt")  # Change to your specific YOLO model
print(model,"_____________________________")
# Input and output video settings
input_video_path = "/home/annone/dataset/VID_20241113_112218.mp4"  # Path to input video or 0 for webcam
output_video_path = "/home/annone/dataset/output_segmented.mp4"
output_fps = 20  # Frames per second for the output video
frame_width = 640
frame_height = 480

# Initialize VideoCapture for input video or camera
cap = cv2.VideoCapture(input_video_path)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

# Check if video is opened
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the input video FPS and frame size if available
input_fps = cap.get(cv2.CAP_PROP_FPS) or output_fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter for output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, input_fps, (frame_width, frame_height))

# Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream or failed to grab frame.")
        break

    # Run YOLO segmentation on the frame
    results = model.predict(frame)
    print(results)
    # Draw segmentation masks on the frame
    annotated_frame = results[0].plot()  # plot will draw the segmented areas on the frame

    # Write annotated frame to output video
    out.write(annotated_frame)

    # Display the processed frame (optional)
    cv2.imshow("Segmented Frame", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_video_path}")

In [ ]:
import cv2
import numpy as np
import base64
from datetime import datetime
from roboflow import Roboflow

# Initialize Roboflow and model
rf = Roboflow(api_key="o9xQlct9Wr77cZXqyV17")
project = rf.workspace().project("muncipal-detect")
model = project.version("3").model

# Video settings
input_video_path = "/home/annone/dataset/VID_20241113_112218.mp4"
output_video_path = "/home/annone/dataset/output_segmented.mp4"
output_fps = 20
frame_width = 640
frame_height = 480

# Initialize VideoCapture
cap = cv2.VideoCapture(input_video_path)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get FPS and frame size
input_fps = cap.get(cv2.CAP_PROP_FPS) or output_fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, input_fps, (frame_width, frame_height))

# Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream or failed to grab frame.")
        break

    try:
        # Encode frame to JPEG bytes and convert to base64 string
        _, buffer = cv2.imencode('.jpg', frame)
        frame_base64 = base64.b64encode(buffer).decode('utf-8')

        # Run model prediction on base64-encoded frame
        predictions = model.predict(frame_base64, confidence=40).json()  # Adjust as needed
        print(predictions)

        # Draw bounding boxes based on predictions
        for prediction in predictions['predictions']:
            x, y, width, height = prediction['x'], prediction['y'], prediction['width'], prediction['height']
            class_name = prediction['class']
            confidence = prediction['confidence']
            
            # Calculate top-left and bottom-right coordinates of bounding box
            start_point = (int(x - width / 2), int(y - height / 2))
            end_point = (int(x + width / 2), int(y + height / 2))
            
            # Draw bounding box and label on frame
            color = (0, 255, 0)  # Green for detected objects
            cv2.rectangle(frame, start_point, end_point, color, 2)
            cv2.putText(frame, f"{class_name} {confidence:.2f}", (start_point[0], start_point[1] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Write annotated frame to output video
        out.write(frame)

        # Display frame (optional)
        cv2.imshow("Segmented Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        print("Error processing frame:", e)
        continue

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_video_path}")


In [ ]:
from inference import InferencePipeline
from inference.core.interfaces.camera.entities import VideoFrame

import cv2
import supervision as sv
import os

# Create a bounding box annotator and label annotator for custom sink
label_annotator = sv.LabelAnnotator()
box_annotator = sv.BoundingBoxAnnotator()

# Define output video path and parameters
output_video_path = "/home/annone/dataset/output_segmented.mp4"
output_fps = 20  # Adjust FPS as needed
frame_width = 640  # Set based on video dimensions or dynamically get from frames
frame_height = 480

# Initialize VideoWriter (Set to None initially, as we will configure it on the first frame)
video_writer = None

def my_custom_sink(predictions: dict, video_frame: VideoFrame):
    global video_writer
    
    # Get the text labels for each prediction
    labels = [p["class"] for p in predictions["predictions"]]
    
    # Load predictions into Supervision Detections API
    detections = sv.Detections.from_inference(predictions)
    
    # Annotate the frame using supervision annotators
    image = label_annotator.annotate(
        scene=video_frame.image.copy(), detections=detections, labels=labels
    )
    image = box_annotator.annotate(image, detections=detections)
    
    # Initialize VideoWriter on the first frame with the correct frame size
    if video_writer is None:
        frame_height, frame_width = image.shape[:2]
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, output_fps, (frame_width, frame_height))
    
    # Write the annotated frame to the output video
    print(detections)
    video_writer.write(image)
    print("writen")
    # Display the annotated image (optional)
    # cv2.imshow("Predictions", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        return  # Exit on 'q' key press

# Initialize the inference pipeline
pipeline = InferencePipeline.init(
    model_id="muncipal-detect/3",
    video_reference="/home/annone/dataset/VID_20241113_112218.mp4",
    on_prediction=my_custom_sink,
    api_key="o9xQlct9Wr77cZXqyV17"
)

# Start and join the pipeline to ensure processing is complete
pipeline.start()
pipeline.join()

# Release resources
if video_writer:
    video_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_video_path}")


In [ ]:
!yolo predict model=yolov8n.pt

In [1]:
import cv2
from ultralytics import YOLO

# Input and output video file paths
input_video_path = '/home/annone/dataset/PXL_20241117_072811852.TS.mp4'
output_video_path = '/home/annone/dataset/garbage_output.mp4'
model2=YOLO(model='/home/annone/ai/survey/data/garbage_weights/best.pt')

# Open the input video using OpenCV
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the VideoWriter to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Make predictions on the frame using the trained YOLO model
    results = model2(frame)

    # Annotate the frame with predictions
    annotated_frame = results[0].plot()

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # (Optional) Display the frame with predictions
    # cv2.imshow('YOLOv8 Predictions', annotated_frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        # break

# Release resources
cap.release()
out.release()
# cv2.destroyAllWindows()

print("Video processing complete. Output saved to:", output_video_path)



0: 384x640 1 garbage, 46.5ms
Speed: 4.7ms preprocess, 46.5ms inference, 112.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 garbage, 4.6ms
Speed: 3.1ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 garbage, 4.4ms
Speed: 1.6ms preprocess, 4.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 garbage, 4.3ms
Speed: 1.7ms preprocess, 4.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 garbage, 4.4ms
Speed: 2.2ms preprocess, 4.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 garbages, 4.8ms
Speed: 1.7ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 garbages, 4.7ms
Speed: 1.5ms preprocess, 4.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 garbage, 4.6ms
Speed: 1.9ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384,

In [ ]:
from ultralytics import YOLO

model = YOLO("/home/annone/ai/survey/data/road2.pt")
results = model.train(data = "/home/annone/ai/survey/data/datasets/muncipal--issue-3/data.yaml", epochs=200, batch=4, device="cuda")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="2cd8Z2IcQY5bffl5ebyw")
project = rf.workspace("objectdetection-xf7fl").project("muncipal-issue")
version = project.version(3)
dataset = version.download("yolov8")

In [ ]:
# changee modified date of a file

from datetime import datetime, timedelta

# Input and output file paths
input_file = "coordinates.txt"
output_file = "coordinates_with_timestamp.txt"

# Start timestamp
start_time = datetime.now()

# Time interval in seconds
interval = 0.5

# Read coordinates from the input file
with open(input_file, "r") as infile:
    lines = infile.readlines()

# Add timestamps to each line
with open(output_file, "w") as outfile:
    for i, line in enumerate(lines):
        # Skip empty lines
        if line.strip():
            # Calculate timestamp
            timestamp = start_time + timedelta(seconds=i * interval)
            # Write the coordinate with timestamp
            outfile.write(f"{line.strip()}, Timestamp: {timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}\n")

print(f"Timestamps added successfully! Output saved to '{output_file}'.")
